# Pruebas de normalidad

Antes de aplicar muchos de los métodos estadísticos (como la prueba t de Student o el ANOVA) es importante verificar si los datos siguien una distribución normal. La literatura especializada registra decenas de pruebas para evaluar la normalidad de los datos. Revisaremos los casos de la la prueba Shapiro-Wilk y la prueba Kolmogorov-Smirnov (con la corrección de Lilliefors). 

En las primeras dos pruebas (y otras más incluidas en este curso), vamos a crear funciones que realicen lo siguiente:
1. *Preparar los datos*: Validar y procesar datos para la prueba.
2. *Realizar la prueba estadística*: Obtener los estadísticos de prueva y valores p.
3. *Mostrar los resultados e interpretación*: Para facilitar la descripción de los resultados de la prueba.
4. *Guardar una conclusión general o el p-valor*: Para facilitar decisiones específicas en flujos automatizados.

In [ ]:
import pandas as pd
import numpy as np

#df = pd.read_excel('../datas/enigh2020.xlsx')
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/data/enigh2020.xlsx')

In [ ]:
# Para propósitos de ejemplo obtenemos una muestra de tamaño mediano
data = df['edad_jefe'].head(100)

## Prueba Shapiro-Wilk
*¿Para qué se utiliza?*
La prueba de Shapiro-Wilk se emplea para evaluar si una muestra de datos proviene de una distribución normal. Es una de las pruebas más potentes para detectar desviaciones de la normalidad. 

*Variables consideradas:*
Se aplica a una única variable cuantitativa (de razón o intervalo), cuyos valores se desean comparar contra una distribución normal teórica.

*¿Cómo funciona?*
La prueba calcula el estadístico W, que compara los valores observados con los que se esperaría bajo una distribución normal, utilizando una combinación lineal de los datos ordenados. El valor de W varía entre 0 y 1, donde un valor cercano a 1 indica que los datos son consistentes con una distribución normal.

*Requisitos o supuestos:*
- La variable debe ser cuantitativa y tener una escala continua o aproximadamente continua.
- Las observaciones deben ser independientes.
- Es más adecuada para muestras pequeñas o moderadas (originalmente diseñada para n < 50, pero se puede aplicar hasta n ≈ 2000).
- Para muestras grandes, puede detectar pequeñas desviaciones de la normalidad que no son relevantes desde el punto de vista práctico. En tales casos, se recomienda complementar con métodos gráficos como histogramas, Q-Q plots o pruebas alternativas como Kolmogórov-Smirnov o Anderson-Darling.

*Hipótesis planteadas:*
- Hipótesis nula (H₀): Los datos provienen de una distribución normal.
- Hipótesis alternativa (H₁): Los datos no provienen de una distribución normal.

*Criterio de decisión:*
Si el valor p asociado al estadístico W es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula y se concluye que los datos no siguen una distribución normal.


In [ ]:
from scipy import stats

def shapiro_wilk_test(data, alpha=0.05):
    """ Aplica una prueba Shapiro-Wilk para evaluar la normalidad de los datos"""
    statistic, p_value = stats.shapiro(data)
    conclusion = "no rechaza" if p_value > alpha else "rechaza"
    print(f'La prueba Shapiro-Wilk {conclusion} la normalidad de los datos (W = {statistic:.4f}, p = {p_value:.4f})')
    return p_value > alpha

In [ ]:
resultado = shapiro_wilk_test(data)

Ejemplo de un reporte de resultados:
>Se aplicó la prueba de normalidad de Shapiro-Wilk a los tiempos de entrega (en horas) de 50 pedidos realizados en la planta de distribución del norte del país. El valor de W fue de 0.942 y el valor-p asociado fue de 0.012, lo que indica que se rechaza la hipótesis nula de normalidad al nivel de significancia del 5%.
Por lo tanto, se concluye que los tiempos de entrega no siguen una distribución normal, lo que sugiere la presencia de asimetrías o valores atípicos en el proceso logístico. Esto implica que para futuros análisis —como la construcción de intervalos de confianza o pruebas de hipótesis— deberán considerarse métodos no paramétricos o transformaciones adecuadas. Además, este hallazgo puede motivar una revisión de los procesos operativos que están generando variabilidad extrema o retrasos puntuales en las entregas.


## Prueba Kolmogorov-Smirnov-Lilliefors
*¿Para qué se utiliza?*
La prueba de Kolmogorov-Smirnov (K-S) se utiliza para evaluar si una muestra de datos cuantitativos sigue una distribución teórica específica (como la normal, exponencial o uniforme). Esta prueba es una herramienta general para comparar la forma de la distribución observada con una distribución acumulada esperada.
Variables consideradas:
Se aplica a una sola variable cuantitativa (de intervalo o razón) cuyos datos se desean contrastar con una distribución acumulativa específica.

*¿Cómo funciona?*
La prueba compara la función de distribución empírica (EDF) de los datos —es decir, la proporción acumulada de valores observados— con la función de distribución acumulada teórica (CDF). Calcula la máxima diferencia absoluta (estadístico D) entre ambas curvas. Cuanto mayor es esta diferencia, mayor es la evidencia contra la hipótesis de que los datos siguen la distribución especificada.

*Requisitos o supuestos:*
- La variable debe ser cuantitativa y continua.
- Las observaciones deben ser independientes.
- La prueba clásica supone que los parámetros de la distribución teórica (como la media y desviación estándar en el caso de la normal) son conocidos de antemano.
- Si los parámetros se estiman a partir de la misma muestra, la prueba original no es estrictamente válida.

*Corrección de Lilliefors:*
Cuando los parámetros de la distribución normal se estiman desde los datos (caso típico en la práctica), se recomienda utilizar la prueba de Lilliefors, una modificación de la K-S diseñada para este caso. Esta corrección ajusta el valor crítico del estadístico para mantener la validez del contraste. En Python, puede aplicarse con la función lilliefors() de la biblioteca statsmodels.

*Hipótesis planteadas:*
- Hipótesis nula (H₀): Los datos provienen de la distribución teórica especificada.
- Hipótesis alternativa (H₁): Los datos no provienen de esa distribución.

*Criterio de decisión:*
Se calcula el estadístico D, que representa la mayor diferencia entre la EDF y la CDF. Si el valor p asociado es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula, concluyendo que los datos no siguen la distribución especificada.


In [ ]:
from statsmodels.stats.diagnostic import lilliefors as sm_lilliefors

def lilliefors_test(data, alpha=0.05):
    """ Aplica una prueba Kolmogorov-Smirnov-Lilliefors para evaluar la normalidad de los datos"""  
    statistic, p_value = sm_lilliefors(data)
    conclusion = "no rechaza" if p_value > alpha else "rechaza"
    print(f'La prueba de Lilliefors {conclusion} la normalidad de los datos (D = {statistic:.4f}, p = {p_value:.4f})')
    return p_value > alpha

In [ ]:
lilliefors_test(data)

Ejemplo de resultados:
>Se aplicó la prueba de normalidad de Kolmogorov-Smirnov con corrección de Lilliefors a los tiempos de espera en el área de carga y descarga (en minutos) registrados durante 30 días hábiles consecutivos. El estadístico D fue de 0.176 y el valor-p correspondiente fue de 0.035, por lo que se rechaza la hipótesis nula de normalidad al nivel de significancia del 5%.

## Prueba Jarque-Bera
La prueba Jarque-Bera (1980) evalúa cuanto se desvían el sesgo y la curtosis observados de los valores esperados bajo una distribución normal. Esta prueba fue desarrollada por el mexicano Carlos M. Jarque junto con Anil K. Bera.
Las hipótesis son:
- Hipótesis nula (H0): Los datos tienen una distribución normal.
- Hipótesis alternativa (H1): Los datos no tienen una distribución normal.

El estadístico Jarque-Bera (JB) sigue aproximadamente una distribución chi-cuadrada con dos grados de libertad asumiendo que la hipótesis nula es verdadera. Este estadístico se incluye en el reporte de resultados de la regresión lineal de la biblioteca de statsmodels.

## Referencias
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html


https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html